# Tarea: Web Scraping con Selenium y Docker
### Objetivo:

Aprender a utilizar Selenium junto con Docker para realizar web scraping y extraer información de una página web. En esta tarea, extraerás los títulos de noticias de una página web de tu elección o la proporcionada.

## Código de Web Scraping

A continuación, te proporciono un script en Python utilizando Selenium para obtener los títulos de noticias de una página web. En este caso, utilizaremos la página de noticias de "https://news.ycombinator.com/" como ejemplo.

Tu debes completar el script.py 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configura las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")  # Ejecutar en modo sin cabeza

# Conéctate al servidor Selenium Grid
driver = webdriver.Remote(
    command_executor="http://selenium-server:4444/wd/hub",
    options=chrome_options
)

try:
    # Abre la página de Hacker News
    driver.get("https://news.ycombinator.com/")
    
    # Espera a que los elementos de los títulos estén presentes
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".titlelink"))
    )
    
    # Extrae los títulos de las noticias
    titles = driver.find_elements(By.CSS_SELECTOR, ".titlelink")
    
    # Imprime los títulos
    print("Títulos de noticias en Hacker News:")
    for index, title in enumerate(titles, 1):
        print(f"{index}. {title.text}")
    
    # Imprime el número total de títulos encontrados
    print(f"\nTotal de títulos encontrados: {len(titles)}")

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    # Cierra el navegador
    driver.quit()
driver.quit()

Descripción del Código:

* Configuración de Selenium: Se configura Selenium con el navegador Chrome utilizando opciones específicas para ejecutarlo dentro de Docker.

* Acceso a la Página Web: El script se conecta a la página de noticias de "YCombinator" y carga el contenido.

* Extracción de Datos: Se utiliza find_elements con la clase storylink para obtener todos los títulos de las noticias en la página.

* Impresión de Resultados: Los títulos extraídos se imprimen en la consola.

### Ejercicio 2:

* Buscar dentro de la página web alguno de los links de arriba (New, Past, etc) y ponerlo en un df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")

driver = webdriver.Remote(
    command_executor="http://selenium-server:4444/wd/hub",
    options=chrome_options
)

try:
    driver.get("https://news.ycombinator.com/")
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topbar a"))
    )
    
    nav_links = driver.find_elements(By.CSS_SELECTOR, ".topbar a")
    
    link_texts = []
    link_hrefs = []
    
    for link in nav_links:
        link_texts.append(link.text)
        link_hrefs.append(link.get_attribute('href'))
    
    df_links = pd.DataFrame({
        'Link Text': link_texts,
        'Link URL': link_hrefs
    })
    
    print("Links de navegación encontrados:")
    print(df_links)
    
    df_links.to_csv('hacker_news_links.csv', index=False)

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    driver.quit()

### Ejercicio 3:

* Buscar algo dentro la pagina en el apartado de "Search" y ponerlo en un df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")  

driver = webdriver.Remote(
    command_executor="http://selenium-server:4444/wd/hub",
    options=chrome_options
)

try:
    driver.get("https://news.ycombinator.com/")
    
    search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q"))
    )
    
    search_term = "Python"
    
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".storylink"))
    )
    
    search_results = driver.find_elements(By.CSS_SELECTOR, ".storylink")
    
    titles = []
    links = []
    subtexts = []
    
    for result in search_results:
        titles.append(result.text)
        links.append(result.get_attribute('href'))
    
    try:
        subtext_elements = driver.find_elements(By.CSS_SELECTOR, ".subtext")
        for subtext in subtext_elements:
            subtexts.append(subtext.text)
    except:
        subtexts = ["No subtext available"] * len(titles)
    
    df_search_results = pd.DataFrame({
        'Title': titles,
        'Link': links,
        'Subtext': subtexts[:len(titles)] 
    })
    
    print(f"Resultados de búsqueda para '{search_term}':")
    print(df_search_results)
    
    df_search_results.to_csv(f'hacker_news_search_{search_term}.csv', index=False)

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    driver.quit()

Ejecución del Script:
    
Para ejecutar el script, asegúrate de que los contenedores de Docker estén corriendo y luego ejecuta el siguiente comando en la terminal:


In [ ]:
docker-compose run --rm python python script.py 

### ENTREGABLES: 
* script.py: El archivo con el código para realizar el web scraping y extraer los títulos de las noticias.
* README.md: Instrucciones para ejecutar el proyecto.